# Chapter 6: Updating and Deleting Data

For a database reset, delete "store.db" and re-run ch04.ipynb.

In [218]:
%run -i 'ch03.ipynb'

2024-03-22 11:50:17,832 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:17,832 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2024-03-22 11:50:17,833 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-22 11:50:17,834 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2024-03-22 11:50:17,834 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-22 11:50:17,835 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customer")
2024-03-22 11:50:17,835 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-22 11:50:17,836 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order")
2024-03-22 11:50:17,836 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-22 11:50:17,836 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("order_detail")
2024-03-22 11:50:17,837 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-22 11:50:17,837 INFO sqlalchemy.engine.Engine COMMIT


Necessary imports for this chapter:

In [219]:
from sqlalchemy import Connection, bindparam, delete, insert, select, update
from sqlalchemy.exc import IntegrityError, SQLAlchemyError, StatementError

Get a connection:

In [220]:
conn = engine.connect()

Function for reducing product inventory:

In [221]:
def decrement_product(
        conn: Connection,
        product_id: int,
        quantity: int = 1,
):
    stmt = (
        update(product)
        .where(product.c.product_id == product_id)
        .values(units_in_stock=product.c.units_in_stock - quantity)
    )
    conn.execute(stmt)
    conn.commit()

Reduce the stock of cameras by one:

In [222]:
decrement_product(conn, 4)  # camera ID = 4

2024-03-22 11:50:17,874 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:17,875 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=(product.units_in_stock - ?) WHERE product.product_id = ?
2024-03-22 11:50:17,876 INFO sqlalchemy.engine.Engine [generated in 0.00200s] (1, 4)
2024-03-22 11:50:17,877 INFO sqlalchemy.engine.Engine COMMIT


Checking the result (with an additional query):

In [223]:
stmt = (
    select(product.c.units_in_stock)
    .where(product.c.product_id == 4)
)
result = conn.scalar(stmt)

2024-03-22 11:50:17,886 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:17,887 INFO sqlalchemy.engine.Engine SELECT product.units_in_stock 
FROM product 
WHERE product.product_id = ?
2024-03-22 11:50:17,887 INFO sqlalchemy.engine.Engine [generated in 0.00160s] (4,)


In [224]:
print(result)

4


Using RETURNING with UPDATE:

In [225]:
def decrement_product_and_return(
    conn: Connection,
    product_id: int,
    quantity: int = 1,
):
    stmt = (
        update(product)
        .where(product.c.product_id == product_id)
        .values(units_in_stock=product.c.units_in_stock - quantity)
        .returning(product.c.product_name, product.c.units_in_stock)
    )

    print("STMT:", stmt)

    result = conn.execute(stmt)
    decremented = result.first()
    conn.commit()

    return decremented

In [226]:
decremented_product = decrement_product_and_return(conn, 4)

STMT: UPDATE product SET units_in_stock=(product.units_in_stock - :units_in_stock_1) WHERE product.product_id = :product_id_1 RETURNING product.product_name, product.units_in_stock
2024-03-22 11:50:17,907 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=(product.units_in_stock - ?) WHERE product.product_id = ? RETURNING product_name, units_in_stock
2024-03-22 11:50:17,907 INFO sqlalchemy.engine.Engine [generated in 0.00094s] (1, 4)
2024-03-22 11:50:17,908 INFO sqlalchemy.engine.Engine COMMIT


In [227]:
print(decremented_product)

('digital camera', 3)


Update multiple rows at once with "executemany":

In [228]:
def restore_inventory(conn: Connection):
    stmt = (
        update(product)
        .where(product.c.product_id == bindparam("id"))
        .values(units_in_stock=bindparam("units"))
    )

    result = conn.execute(
        stmt,
        [
            {"id": 1, "units": 5},
            {"id": 2, "units": 10},
            {"id": 3, "units": 10},
            {"id": 4, "units": 5},
            {"id": 5, "units": 1},
        ],
    )
    conn.commit()
    print(f"Matching rows: {result.rowcount}")

In [229]:
restore_inventory(conn)

2024-03-22 11:50:17,927 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:17,928 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=? WHERE product.product_id = ?
2024-03-22 11:50:17,928 INFO sqlalchemy.engine.Engine [generated in 0.00111s] [(5, 1), (10, 2), (10, 3), (5, 4), (1, 5)]
2024-03-22 11:50:17,929 INFO sqlalchemy.engine.Engine COMMIT
Matching rows: 5


## Common Exceptions and Error Handling

IntegrityError:

In [230]:
def update_exception(conn: Connection):
    stmt = update(product).values(units_in_stock=-1)
    conn.execute(stmt)

This will result in an error (uncomment to see):

In [231]:
# update_exception(conn)

Adding error handling:

In [232]:
try:
    update_exception(conn)
except SQLAlchemyError as e:
    print(f"Some {type(e).__name__} occurred!")
    print(e)

2024-03-22 11:50:17,945 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:17,946 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=?
2024-03-22 11:50:17,946 INFO sqlalchemy.engine.Engine [generated in 0.00096s] (-1,)
Some IntegrityError occurred!
(sqlite3.IntegrityError) CHECK constraint failed: units_in_stock>=0
[SQL: UPDATE product SET units_in_stock=?]
[parameters: (-1,)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


The `IntegrityError` class extends `StatementError`, so you can be more specific and do the following:

In [233]:
try:
    update_exception(conn)
except StatementError as e:
    print("# StatementError type:", type(e).__name__)
    print("statement:", e.statement)
    print("params:", e.params)
    print("orig:", e.orig)
except SQLAlchemyError as e:
    print("# Exception type:", type(e).__name__)
    print(e)

2024-03-22 11:50:17,952 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=?
2024-03-22 11:50:17,952 INFO sqlalchemy.engine.Engine [cached since 0.006846s ago] (-1,)
# StatementError type: IntegrityError
statement: UPDATE product SET units_in_stock=?
params: (-1,)
orig: CHECK constraint failed: units_in_stock>=0


Of course, you can handle the `IntegrityError` directly:

In [234]:
try:
    update_exception(conn)
except IntegrityError as e:
    print('An IntegrityError occurred!')
    # TODO: handle your integrity errors here
except StatementError as e:
    print("# StatementError type:", type(e).__name__)
    print("statement:", e.statement)
    print("params:", e.params)
    print("orig:", e.orig)
except SQLAlchemyError as e:
    print("# Exception type:", type(e).__name__)
    print(e)

2024-03-22 11:50:17,958 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=?
2024-03-22 11:50:17,958 INFO sqlalchemy.engine.Engine [cached since 0.01281s ago] (-1,)
An IntegrityError occurred!


To reuse this connection, you need to rollback the current transaction:

In [235]:
conn.rollback() # necessary if you want to reuse the same connection

2024-03-22 11:50:17,963 INFO sqlalchemy.engine.Engine ROLLBACK


Closing the connection:

In [236]:
conn.close()

## Transactions

commit as you go:

In [237]:
with engine.connect() as conn:
    stmt = insert(employee).values(name="Amelia")
    conn.execute(stmt)  # transaction 1 begins automatically
    conn.commit()  # transaction 1 committed, Amelia is hired!

    stmt = insert(employee).values(name="Brian")
    conn.execute(stmt)  # new transaction 2 begins
    conn.rollback()  # transaction 2 is rolled back

    stmt = insert(employee).values(name="Charlotte")
    conn.execute(stmt)  # another new transaction 3 begins
    conn.commit()  # transaction 3 committed, Charlotte is hired!

2024-03-22 11:50:17,972 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:17,973 INFO sqlalchemy.engine.Engine INSERT INTO employee (name, is_manager, hire_date) VALUES (?, ?, ?)
2024-03-22 11:50:17,973 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ('Amelia', 0, '2024-03-22')
2024-03-22 11:50:17,974 INFO sqlalchemy.engine.Engine COMMIT
2024-03-22 11:50:17,976 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:17,977 INFO sqlalchemy.engine.Engine INSERT INTO employee (name, is_manager, hire_date) VALUES (?, ?, ?)
2024-03-22 11:50:17,977 INFO sqlalchemy.engine.Engine [cached since 0.005331s ago] ('Brian', 0, '2024-03-22')
2024-03-22 11:50:17,978 INFO sqlalchemy.engine.Engine ROLLBACK
2024-03-22 11:50:17,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:17,979 INFO sqlalchemy.engine.Engine INSERT INTO employee (name, is_manager, hire_date) VALUES (?, ?, ?)
2024-03-22 11:50:17,980 INFO sqlalchemy.engine.Engine [cached since 0.00799s ago]

Begin once (`Engine.begin()`):

In [238]:
with engine.begin() as conn:
    stmt = insert(employee).values(name="Daniel")
    conn.execute(stmt)
    stmt = insert(employee).values(name="Emily")
    conn.execute(stmt)
    # the transaction is committed implicitly at the end of this block

2024-03-22 11:50:17,988 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:17,988 INFO sqlalchemy.engine.Engine INSERT INTO employee (name, is_manager, hire_date) VALUES (?, ?, ?)
2024-03-22 11:50:17,989 INFO sqlalchemy.engine.Engine [cached since 0.01683s ago] ('Daniel', 0, '2024-03-22')
2024-03-22 11:50:17,990 INFO sqlalchemy.engine.Engine INSERT INTO employee (name, is_manager, hire_date) VALUES (?, ?, ?)
2024-03-22 11:50:17,990 INFO sqlalchemy.engine.Engine [cached since 0.01789s ago] ('Emily', 0, '2024-03-22')
2024-03-22 11:50:17,990 INFO sqlalchemy.engine.Engine COMMIT


Example: Order Processing

In [239]:
conn = engine.connect()

In [240]:
def process_order(conn: Connection, order_id: int):
    print(f"# Processing order {order_id}.")

    # check if the order is already processed
    stmt = (
        select(order.c.is_shipped)
        .where(order.c.order_id == order_id)
    )
    is_shipped = conn.scalar(stmt)
    if is_shipped:
        print("The order is already shipped.")
        return

    # order is not shipped, process it
    # get the order details: (product_id, quantity)
    stmt = (
        select(order_detail.c.product_id, order_detail.c.quantity)
        .where(order_detail.c.order_id == order_id)
    )
    result = conn.execute(stmt)

    # process each product ordered
    update_success = True
    for product_id, quantity in result:
        print(f"Processing product {product_id} x{quantity}.")
        try:
            stmt_update = (
                update(product)
                .where(product.c.product_id == product_id)
                # you can also check for negative values
                .values(units_in_stock=product.c.units_in_stock - quantity)
            )
            conn.execute(stmt_update)
        except IntegrityError as e:
            print("An error occurred while updating product's units in stock!")
            print(e.orig)

            # to continue using the same connection for further operations,
            # roll back the current transaction
            print("Rolling back transaction...")
            conn.rollback()

            update_success = False
            break

    if update_success:
        stmt_update = (
            update(order)
            .where(order.c.order_id == order_id)
            .values(is_shipped=True)
        )
        conn.execute(stmt_update)

        # commit explicitly in commit-as-you-go
        conn.commit()

In [241]:
process_order(conn, 1)  # success

# Processing order 1.
2024-03-22 11:50:18,008 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:18,009 INFO sqlalchemy.engine.Engine SELECT "order".is_shipped 
FROM "order" 
WHERE "order".order_id = ?
2024-03-22 11:50:18,009 INFO sqlalchemy.engine.Engine [generated in 0.00127s] (1,)
2024-03-22 11:50:18,010 INFO sqlalchemy.engine.Engine SELECT order_detail.product_id, order_detail.quantity 
FROM order_detail 
WHERE order_detail.order_id = ?
2024-03-22 11:50:18,010 INFO sqlalchemy.engine.Engine [generated in 0.00030s] (1,)
Processing product 1 x1.
2024-03-22 11:50:18,011 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=(product.units_in_stock - ?) WHERE product.product_id = ?
2024-03-22 11:50:18,011 INFO sqlalchemy.engine.Engine [cached since 0.1372s ago] (1, 1)
Processing product 2 x1.
2024-03-22 11:50:18,012 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=(product.units_in_stock - ?) WHERE product.product_id = ?
2024-03-22 11:50:18,012 INFO 

In [242]:
process_order(conn, 2)  # success

# Processing order 2.
2024-03-22 11:50:18,021 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:18,021 INFO sqlalchemy.engine.Engine SELECT "order".is_shipped 
FROM "order" 
WHERE "order".order_id = ?
2024-03-22 11:50:18,021 INFO sqlalchemy.engine.Engine [cached since 0.01361s ago] (2,)
2024-03-22 11:50:18,022 INFO sqlalchemy.engine.Engine SELECT order_detail.product_id, order_detail.quantity 
FROM order_detail 
WHERE order_detail.order_id = ?
2024-03-22 11:50:18,023 INFO sqlalchemy.engine.Engine [cached since 0.01267s ago] (2,)
Processing product 5 x1.
2024-03-22 11:50:18,023 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=(product.units_in_stock - ?) WHERE product.product_id = ?
2024-03-22 11:50:18,024 INFO sqlalchemy.engine.Engine [cached since 0.15s ago] (1, 5)
2024-03-22 11:50:18,024 INFO sqlalchemy.engine.Engine UPDATE "order" SET is_shipped=? WHERE "order".order_id = ?
2024-03-22 11:50:18,025 INFO sqlalchemy.engine.Engine [cached since 0.01114s ago]

In [243]:
process_order(conn, 3)  # this will fail

# Processing order 3.
2024-03-22 11:50:18,032 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:18,032 INFO sqlalchemy.engine.Engine SELECT "order".is_shipped 
FROM "order" 
WHERE "order".order_id = ?
2024-03-22 11:50:18,033 INFO sqlalchemy.engine.Engine [cached since 0.02471s ago] (3,)
2024-03-22 11:50:18,033 INFO sqlalchemy.engine.Engine SELECT order_detail.product_id, order_detail.quantity 
FROM order_detail 
WHERE order_detail.order_id = ?
2024-03-22 11:50:18,034 INFO sqlalchemy.engine.Engine [cached since 0.02387s ago] (3,)
Processing product 4 x1.
2024-03-22 11:50:18,035 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=(product.units_in_stock - ?) WHERE product.product_id = ?
2024-03-22 11:50:18,035 INFO sqlalchemy.engine.Engine [cached since 0.161s ago] (1, 4)
Processing product 5 x2.
2024-03-22 11:50:18,036 INFO sqlalchemy.engine.Engine UPDATE product SET units_in_stock=(product.units_in_stock - ?) WHERE product.product_id = ?
2024-03-22 11:50:18,03

Function for printing order and product status:

In [244]:
def print_order_and_product_status(conn: Connection):
    print("# Order status:")
    stmt = (
        select(order.c.order_id, order.c.is_shipped)
        .order_by(order.c.order_id)
    )
    for row in conn.execute(stmt):
        print(f"Order#{row.order_id}, shipped: {row.is_shipped}")

    print("# Product status:")
    stmt = (
        select(
            product.c.product_id,
            product.c.product_name,
            product.c.units_in_stock
        )
        .order_by(product.c.product_id)
    )
    for row in conn.execute(stmt):
        print(f"Product#{row[0]} {row[1]}: {row[2]}")

In [245]:
print_order_and_product_status(conn)

# Order status:
2024-03-22 11:50:18,046 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:18,047 INFO sqlalchemy.engine.Engine SELECT "order".order_id, "order".is_shipped 
FROM "order" ORDER BY "order".order_id
2024-03-22 11:50:18,047 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
Order#1, shipped: True
Order#2, shipped: True
Order#3, shipped: False
# Product status:
2024-03-22 11:50:18,048 INFO sqlalchemy.engine.Engine SELECT product.product_id, product.product_name, product.units_in_stock 
FROM product ORDER BY product.product_id
2024-03-22 11:50:18,048 INFO sqlalchemy.engine.Engine [generated in 0.00026s] ()
Product#1 phone: 4
Product#2 phone screen protector: 9
Product#3 headphone: 9
Product#4 digital camera: 5
Product#5 memory card 256GB: 0


## Deleting Data


In [246]:
def delete_employee(conn: Connection, name: str):
    stmt = (
        delete(employee)
        .where(employee.c.name == name)
    )

    result = conn.execute(stmt)
    print("Employees selected for deletion:", result.rowcount)

    conn.commit()

In [247]:
delete_employee(conn, "Amelia")

2024-03-22 11:50:18,057 INFO sqlalchemy.engine.Engine DELETE FROM employee WHERE employee.name = ?
2024-03-22 11:50:18,058 INFO sqlalchemy.engine.Engine [generated in 0.00054s] ('Amelia',)
Employees selected for deletion: 1
2024-03-22 11:50:18,059 INFO sqlalchemy.engine.Engine COMMIT


Using RETURNING with DELETE:

In [248]:
stmt = (
    delete(employee)
    .where(
        employee.c.is_manager == False,
        employee.c.manager_id == None,
    )
    .returning(
        employee.c.employee_id,
        employee.c.name,
    )
)

In [249]:
print(stmt)

DELETE FROM employee WHERE employee.is_manager = false AND employee.manager_id IS NULL RETURNING employee.employee_id, employee.name


In [250]:
result = conn.execute(stmt)
rows = result.all()
print("Employees selected for deletion:", len(rows))  # rowcount doesn't work for SQLite3, so...

2024-03-22 11:50:18,074 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-22 11:50:18,074 INFO sqlalchemy.engine.Engine DELETE FROM employee WHERE employee.is_manager = 0 AND employee.manager_id IS NULL RETURNING employee_id, name
2024-03-22 11:50:18,075 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ()
Employees selected for deletion: 4


In [251]:
for row in result:
    print(row)

conn.commit()

2024-03-22 11:50:18,079 INFO sqlalchemy.engine.Engine COMMIT


Remember to close the connection, as always:

In [252]:
conn.close()